In [ ]:
from fastai.conv_learner import *
from fastai.dataset import *

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import random
import math
import imgaug as ia
from imgaug import augmenters as iaa

In [ ]:
PATH = '/home/baitong/pywork/Humpback/Data/'
TRAIN = PATH+'train/'
TEST = PATH+'test/'
LABELS = PATH+'train.csv'
BOXES = PATH+'bounding_boxes.csv'
MODLE_INIT = PATH+'models/'

n_embedding = 256
bs = 16
ratio = 3
sz0 = 192
sz = (ratio*sz0,sz0)
nw = 2

In [ ]:
def open_image(fn):
    flags = cv2.IMREAD_UNCHANGED+cv2.IMREAD_ANYDEPTH+cv2.IMREAD_ANYCOLOR
    if not os.path.exists(fn):
        raise OSError('No such file or directory: {}'.format(fn))
    elif os.path.isdir(fn):
        raise OSError('Is a directory: {}'.format(fn))
    else:
        try:
            im = cv2.imread(str(fn), flags)
            if im is None: raise OSError(f'File not recognized by opencv: {fn}')
            return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        except Exception as e:
            raise OSError('Error handling image at: {}'.format(fn)) from e

class Loader():
    def __init__(self,path,tfms_g=None, tfms_px=None):
        #tfms_g - geometric augmentation (distortion, small rotation, zoom)
        #tfms_px - pixel augmentation and flip
        self.boxes = pd.read_csv(BOXES).set_index('Image')
        self.path = path
        self.tfms_g = iaa.Sequential(tfms_g,random_order=False) \
                        if tfms_g is not None else None
        self.tfms_px = iaa.Sequential(tfms_px,random_order=False) \
                        if tfms_px is not None else None
    def __call__(self, fname):
        fname = os.path.basename(fname)
        x0,y0,x1,y1 = tuple(self.boxes.loc[fname,['x0','y0','x1','y1']].tolist())
        img = open_image(os.path.join(self.path,fname))
        if self.tfms_g != None: img = self.tfms_g.augment_image(img)
        l1,l0,_ = img.shape
        b0,b1 = x1-x0 + 50, y1-y0 + 20 #add extra paddning
        b0n,b1n = (b0, b0/ratio) if b0**2/ratio > b1**2*ratio else (b1*ratio, b1)
        if b0n > l0: b0n,b1n = l0,b1n*l0/b0n
        if b1n > l1: b0n,b1n = b0n*l1/b1n,l1
        x0n = (x0 + x1 - b0n)/2
        x1n = (x0 + x1 + b0n)/2
        y0n = (y0 + y1 - b1n)/2
        y1n = (y0 + y1 + b1n)/2
        x0n,x1n,y0n,y1n = int(x0n),int(x1n),int(y0n),int(y1n)
        if(x0n < 0): x0n,x1n = 0,x1n-x0n
        elif(x1n > l0): x0n,x1n = x0n+l0-x1n,l0
        if(y0n < 0): y0n,y1n = 0,y1n-y0n
        elif(y1n > l1): y0n,y1n = y0n+l1-y1n,l1
        img = cv2.resize(img[y0n:y1n,x0n:x1n,:], sz)
        if self.tfms_px != None: img = self.tfms_px.augment_image(img)
        return img.astype(np.float)/255

In [ ]:
class pdFilesDataset(FilesDataset):
    def __init__(self, data, path, transform):
        df = data.copy()
        counts = Counter(df.Id.values)
        df['c'] = df['Id'].apply(lambda x: counts[x])
        #in the production runs df.c>1 should be used
        fnames = df[(df.c>2) & (df.Id != 'new_whale')].Image.tolist()
        df['label'] = df.Id
        df.loc[df.c == 1,'label'] = 'new_whale'
        df = df.sort_values(by=['c'])
        df.label = pd.factorize(df.label)[0]
        l1 = 1 + df.label.max()
        l2 = len(df[df.label==0])
        df.loc[df.label==0,'label'] = range(l1, l1+l2) #assign unique ids
        self.labels = df.copy().set_index('Image')
        self.names = df.copy().set_index('label')
        if path == TRAIN:
            #data augmentation: 8 degree rotation, 10% stratch, shear
            tfms_g = [iaa.Affine(rotate=(-8, 8),mode='reflect',
                scale={"x": (0.9, 1.1), "y": (0.9, 1.1)}, shear=(-8,8))]
            #data augmentation: horizontal flip, hue and staturation augmentation,
            #gray scale, blur
            tfms_px = [iaa.Fliplr(0.5), iaa.AddToHueAndSaturation((-20, 20)),
                iaa.Grayscale(alpha=(0.0, 1.0)),iaa.GaussianBlur((0, 1.0))]
            self.loader = Loader(path,tfms_g,tfms_px)
        else: self.loader = Loader(path)
        super().__init__(fnames, transform, path)
    
    def get_x(self, i):
        label = self.labels.loc[self.fnames[i],'label']
        #random selection of a positive example
        for j in range(10): #sometimes loc call fails
            try:
                names = self.names.loc[label].Image
                break
            except: None
        name_p = names if isinstance(names,str) else \
            random.sample(set(names) - set([self.fnames[i]]),1)[0]
        #random selection of a negative example
        for j in range(10): #sometimes loc call fails
            try:
                names = self.names.loc[self.names.index!=label].Image
                break
            except: None
        name_n = names if isinstance(names,str) else names.sample(1).values[0]
        imgs = [self.loader(os.path.join(self.path,self.fnames[i])),
                self.loader(os.path.join(self.path,name_p)),
                self.loader(os.path.join(self.path,name_n)),
                label,label,self.labels.loc[name_n,'label']]
        return imgs
    
    def get_y(self, i):
        return 0
    
    def get(self, tfm, x, y):
        if tfm is None:
            return (*x,0)
        else:
            x1, y1 = tfm(x[0],x[3])
            x2, y2 = tfm(x[1],x[4])
            x3, y3 = tfm(x[2],x[5])
            #combine all images into one tensor
            x = np.stack((x1,x2,x3),0)
            return x,(y1,y2,y3)
        
    def get_names(self,label):
        names = []
        for j in range(10):
            try:
                names = self.names.loc[label].Image
                break
            except: None
        return names
        
    @property
    def is_multi(self): return True
    @property
    def is_reg(self):return True
    
    def get_c(self): return n_embedding
    def get_n(self): return len(self.fnames)
    
#class for loading an individual images when embedding is computed
class FilesDataset_single(FilesDataset):
    def __init__(self, data, path, transform):
        self.loader = Loader(path)
        fnames = os.listdir(path)
        super().__init__(fnames, transform, path)
        
    def get_x(self, i):
        return self.loader(os.path.join(self.path,self.fnames[i]))
                           
    def get_y(self, i):
        return 0
        
    @property
    def is_multi(self): return True
    @property
    def is_reg(self):return True
    
    def get_c(self): return n_embedding
    def get_n(self): return len(self.fnames)

In [ ]:
def get_data(sz,bs):
    tfms = tfms_from_model(resnet34, sz, crop_type=CropType.NO)
    tfms[0].tfms = [tfms[0].tfms[2],tfms[0].tfms[3]]
    tfms[1].tfms = [tfms[1].tfms[2],tfms[1].tfms[3]]
    df = pd.read_csv(LABELS)
    trn_df, val_df = train_test_split(df,test_size=0.2, random_state=42)
    ds = ImageData.get_ds(pdFilesDataset, (trn_df,TRAIN), (val_df,TRAIN), tfms)
    md = ImageData(PATH, ds, bs, num_workers=nw, classes=None)
    return md

In [ ]:
md = get_data(sz,bs)

x,y = next(iter(md.trn_dl))
print(x.shape, y[0].shape)

def display_imgs(x):
    columns = 3
    rows = min(bs,16)
    fig=plt.figure(figsize=(columns*8, rows*3))
    for i in range(rows):
        for j in range(columns):
            idx = j+i*columns
            fig.add_subplot(rows, columns, idx+1)
            plt.axis('off')
            plt.imshow((x[j][i,:,:,:]*255).astype(np.int))
    plt.show()
    
display_imgs((md.trn_ds.denorm(x[:,0,:,:,:]),md.trn_ds.denorm(x[:,1,:,:,:]),md.trn_ds.denorm(x[:,2,:,:,:])))

In [ ]:
def resnext50(pretrained=True):
    model = resnext_50_32x4d()
    name = 'resnext_50_32x4d.pth'
    if pretrained:
        path = os.path.join(MODLE_INIT,name)
        load_model(model, path)
    return model

class TripletResneXt50(nn.Module):
    def __init__(self, pre=True, emb_sz=64, ps=0.5):
        super().__init__()
        encoder = resnext50(pretrained=pre)
        self.cnn = nn.Sequential(encoder[0],encoder[1],nn.ReLU(),encoder[3],
                        encoder[4],encoder[5],encoder[6],encoder[7])
        self.head = nn.Sequential(AdaptiveConcatPool2d(), Flatten(), nn.Dropout(ps),
                        nn.Linear(4096, 512), nn.ReLU(), nn.BatchNorm1d(512),
                        nn.Dropout(ps), nn.Linear(512, emb_sz))
        
    def forward(self,x):
        x1,x2,x3 = x[:,0,:,:,:],x[:,1,:,:,:],x[:,2,:,:,:]
        x1 = self.head(self.cnn(x1))
        x2 = self.head(self.cnn(x2))
        x3 = self.head(self.cnn(x3))
        return torch.cat((x1.unsqueeze_(-1),x2.unsqueeze_(-1),x3.unsqueeze_(-1)),dim=-1)
    
    def get_embedding(self, x):
        return self.head(self.cnn(x))
    
class ResNeXt50Model():
    def __init__(self,pre=True,name='TripletResneXt50',**kwargs):
        self.model = to_gpu(TripletResneXt50(pre=True,**kwargs))
        self.name = name

    def get_layer_groups(self, precompute):
        m = self.model.module if isinstance(self.model,FP16) else self.model
        if precompute:
            return [m.head]
        c = children(m.cnn)
        return list(split_by_idxs(c,[5])) + [m.head]

In [ ]:
def Contrastive_loss(preds, target, size_average=True, m=10.0):
    #matrix of all vs all comparisons
    t = torch.cat(target)
    sz = t.shape[0]
    t1 = t.unsqueeze(1).expand((sz,sz))
    t2 = t1.transpose(0,1)
    y = t1==t2
    
    pred = torch.cat((preds[:,:,0], preds[:,:,1], preds[:,:,2]))
    half = True if isinstance(pred,torch.cuda.HalfTensor) else False
    if half : pred = pred.float()
    pred1 = pred.unsqueeze(1).expand((sz,sz,-1))
    pred2 = pred1.transpose(0,1)
    d = (pred1 - pred2).pow(2).sum(dim=-1)
    loss_p = d[y==1]
    loss_n = F.relu(m - torch.sqrt(d[y==0]))**2
    loss = torch.cat((loss_p,loss_n),0)
    loss = loss.mean() if size_average else loss.sum()
    if half : pred = pred.half()
    return loss

def DB_acc(preds, target):
    v, p, n = preds[:,:,0], preds[:,:,1], preds[:,:,2]
    dp = (p - v).pow(2).sum(dim=1)
    dn = (n - v).pow(2).sum(dim=1)
    return (dp < dn).float().mean()

In [ ]:
learner = ConvLearner(md,ResNeXt50Model(ps=0.0,emb_sz=n_embedding))
learner.opt_fn = optim.Adam
learner.clip = 1.0 #gradient clipping
learner.crit = Contrastive_loss
learner.metrics = [DB_acc]
learner #click "output" to see details of the model

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    learner.lr_find()
learner.sched.plot()

In [ ]:
lr = 5e-4
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    learner.fit(lr,1)

In [ ]:
learner.unfreeze()
lrs=np.array([lr/25,lr/5,lr])
# learner.half() #

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    learner.fit(lrs/4,2,cycle_len=1,use_clr=(10,20))
    learner.fit(lrs/8,4,cycle_len=2,use_clr=(10,20))
    learner.fit(lrs/16,8,cycle_len=2,use_clr=(10,20)) 

In [ ]:
learner.fit(lrs/32,8,cycle_len=2,use_clr=(10,20)) 
